In [2]:
# Importanción de librerías.
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import requests

In [3]:
# Credenciales de Spotify.
CLIENT_ID = 'a018711755a146b2a678a93741dc6041'
CLIENT_SECRET = '51fa44e4b57343f19c810b9a75089da6'

# Autenticación con Spotify.
auth_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [4]:
# Búsqueda en la API de Spotify a través de la librería spotipy.
datos = sp.search(q="genre:rock year:2020", type='track', limit=50, offset=0)

In [5]:
# Géneros y rango de años elegidos.
genres = ["pop", "rock", "jazz", "hip hop"]
year_start = 2002
year_end = 2007

# Almacenar los resultados de canciones y álbumes en listas.
lista_canciones_albumes = [] 
lista_artistas = []

In [6]:
# Acceder a los detalles del álbum de la primera canción que aparece en la lista de resultados que te devuelve la API de Spotify.
datos['tracks']['items'][0]['album']['type']


'album'

In [27]:
# Bucle para recorrer los géneros de la lista 'genres'.
for genre in genres: 
    # Realizar la búsqueda en la API de Spotify, buscando canciones (tracks) y álbumes (albums).
    # Se busca por género y por rango de años, limitando los resultados a 50 y estableciendo un desplazamiento (offset) de 951.
    datos = sp.search(q=f"genre:{genre} year:{year_start}-{year_end}", type='track,album', limit=50, offset=901)

    # Recorremos la lista de canciones encontradas en los resultados de la búsqueda.
    for item in datos['tracks']['items']: # 'tracks' contiene las canciones encontradas.
        
        # Agregamos, para cada canción, un diccionario con detalles relevantes a la lista.
        lista_canciones_albumes.append(
            {
                "nombre_cancion":item['name'],
                "nombre_artista":item['artists'][0]['name'],
                'genre':genre,                 
                "type":  item['album']['type'],
                "year": item['album']['release_date'][:4],
                "id": item['id']
            }
        )

        # Agregamos el nombre del artista a la lista.
        lista_artistas.append(
            {
                "nombre_artista":item['artists'][0]['name'],
            }
        )

In [28]:
# Longitud de la lista de artistas.
len(lista_artistas)

3800

In [29]:
# Longitud de la lista de canciones_albumes.
len(lista_canciones_albumes)

3800

In [30]:
# Convierte la lista de diccionarios en un DataFrame de Pandas.
df_lista_artistas = pd.DataFrame(lista_artistas)

In [31]:
# Limpia la lista lista_artistas. Si un artista aparece más de una vez en la columna, ahora solo aparecerá una vez en el conjunto.
lista_artistas_limpia = set(df_lista_artistas["nombre_artista"])


In [32]:
# Convierte la lista de diccionarios en un DataFrame de Pandas.
df_lista_canciones_albumes = pd.DataFrame(lista_canciones_albumes)

# Guarda el DataFrame de canciones en un archivo CSV, eliminando el índice, para que los datos puedan ser utilizados más fácilmente fuera de Python.
df_lista_canciones_albumes.to_csv("canciones.csv", index= False)


In [34]:
# Convierte la lista de diccionarios en un DataFrame de Pandas.
df_artistas_limpia = pd.DataFrame(lista_artistas_limpia)

# Guarda el DataFrame de artistas en un archivo CSV, eliminando el índice, para que los datos puedan ser utilizados más fácilmente fuera de Python.
df_artistas_limpia.to_csv("artistas.csv", index = False)

In [35]:
# Como ya tenemos los dos archivos CSV con la información de artistas, canciones y álbumes, desde la API de last.fm sacamos la información de cada artista.

# Credenciales de last.fm.
API_KEY = 'bb92344bdb091e7c718014c1dcd0e28d'  
BASE_URL = 'http://ws.audioscrobbler.com/2.0/'

In [36]:
# Declaración nueva lista para posteriormente añadir os datos
datos_artistas = []

In [37]:
# Bucle para recorrer la lista de artistas ya limpia.
for artista in lista_artistas_limpia:

    # Parámetros para hacer una solicitud a la API.
    params_info = {
        "method": "artist.getInfo", 
        "api_key": API_KEY,
        "format": "json", 
        "artist": artista
    }
    try:
        # Consulta de la API. Realizamos la solicitud GET a la API con los parámetros definidos.
        response_info = requests.get(BASE_URL, params = params_info)

        # Verificamos si la solicitud fue exitosa (código de estado 200), sino se imprime un mensaje de error con el nombre del artista y el código de estado.
        if response_info.status_code !=200:
            print(f"Error en la solicitud para {artista}: {response_info.status_code}")
            continue

        # Si la solicitud fue exitosa, convertimos la respuesta en formato JSON.
        data_json = response_info.json()

        # Extraemos la información del artista del JSON de la respuesta.
        artist_data = data_json.get('artist', {})

        # Extraemos el resumen biográfico del artista. Si no está disponible, se asigna "Información no disponible".
        resumen = artist_data.get('bio', {}).get('summary', "Información no disponible")

        # Extraemos la popularidad (número de oyentes). Si no está disponible, se asigna "No disponible".
        popularidad = artist_data.get('stats', {}).get('listeners', "No disponible")

        # Extraemos el número de reproducciones (playcount). Si no está disponible, se asigna "No disponible".
        reproducciones = artist_data.get('stats', {}).get('playcount', "No disponible")

        # Extraemos los artistas similares. Si no hay artistas similares, se asigna una lista vacía.
        similares = [sim['name'] for sim in artist_data.get('similar', {}).get('artist', [])]

        # Añadimos un diccionario con la información del artista a la lista declarada anteriormente.
        # El diccionario contiene el nombre del artista, el resumen biográfico, la popularidad, el número de reproducciones y los artistas similares.
        datos_artistas.append({"artista": artista, 
                               "resumen": resumen,
                                "popularidad": popularidad,
                                "reproducciones": reproducciones,
                                "similares": similares})
    
    # Si ocurre un error debido a que falta alguna clave en el JSON capturamos el KeyError y mostramos un mensaje indicando que falta información para ese artista.
    except KeyError:
        print(f"Falta información para el artista: {artista}")

    # Si ocurre cualquier otro error inesperado (por ejemplo, problemas de red o conexión con la API), capturamos la excepción general y mostramos el mensaje de error junto con el nombre del artista.
    except Exception as e:
        print(f"Ocurrió un error inesperado con {artista}: {e}")
        
    

In [38]:
# Convierte la lista de diccionarios en un DataFrame de Pandas.
df_datos_artistas = pd.DataFrame(datos_artistas)

# Guarda el DataFrame de datos_artistas en un archivo CSV, eliminando el índice, para que los datos puedan ser utilizados más fácilmente fuera de Python.
df_datos_artistas.to_csv("datos_artistas.csv", index = False)

In [61]:
# Conectamos Python con MySQL. 
# Importación de módulos en Python que se utilizan para interactuar con bases de datos MySQL, manejar errores y trabajar con datos.
import mysql.connector
from mysql.connector import errorcode
import pandas as pd


In [62]:
# Leectura del archivo 'canciones.csv' para cargarlo en un DataFrame de pandas y poder así trabajar con los datos de las canciones como si fuera una tabla.
df_canciones = pd.read_csv('canciones.csv')
df_canciones

,nombre_cancion,nombre_artista,genre,type,year,id
0,Caminando por la vida,Melendi,pop,album,2005,3v1dCP3hk2djfWryqfp7sx
1,Por la boca vive el pez,Fito y Fitipaldis,pop,album,2006,24UStI5AYA7g5ufFDh3LH0
2,Soldadito marinero,Fito y Fitipaldis,pop,album,2004,0eVborSuxUeSg0meWYd9dZ
3,No Puedo Vivir Sin Ti,Los Ronaldos,pop,album,2007,1EC0JRoSi9YL5zExd3Euc3
4,Rosas,La Oreja de Van Gogh,pop,album,2003,4waqcUQWdj0yH26STWl2Rq
...,...,...,...,...,...,...
3795,Red White Blue,Abstrakt Intellekt,hip hop,album,2007,5SOZwDQqYj7Wjh6KbvbOCp
3796,Voice of a Generation,Abstrakt Intellekt,hip hop,album,2007,2GPL1rrLQ4N5spa6gBktj2
3797,Le Hip-Hop mon royaume,Rocca,hip hop,album,2002,5kp38Y1j7CEgqmYExn3syW
3798,Stupid (feat. Katey Red),Big Freedia,hip hop,album,2002,1Bt01zWszIppYc5PHJ0oeE


In [63]:
# Leectura del archivo 'datos_artistas.csv' para cargarlo en un DataFrame de pandas y poder así trabajar con los datos de las canciones como si fuera una tabla.
df_artistas = pd.read_csv('datos_artistas.csv')
df_artistas

,artista,resumen,popularidad,reproducciones,similares
0,Nina Simone,Eunice Kathleen Waymon (21 February 1933 – 21 ...,2587613,61048146,"['Aretha Franklin', 'Billie Holiday', 'Etta Ja..."
1,Chevelle,Chevelle are an alternative metal band from Gr...,1430244,41038478,"['10 Years', 'Staind', 'Seether', 'Breaking Be..."
2,Elis Regina,"Elis Regina Carvalho Costa (March 17, 1945, Po...",842671,20463391,"['Gal Costa', 'Antonio Carlos Jobim & Elis Reg..."
3,Fata El Presidente,"<a href=""https://www.last.fm/music/Fata+El+Pr...",246,639,"['Viviane Chidid', 'Aïda Samb', 'Pape Diouf', ..."
4,Julie London,"Julie London (née Peck; September 26,1926 — O...",1087070,11683452,"['Peggy Lee', 'Doris Day', 'June Christy', 'Sa..."
...,...,...,...,...,...
1399,Katya Lel,Катя Лель was born on September 20th 1974 and ...,194757,2271072,"['Britney Manson', 'Erika Lundmoen', ""Aliyah's..."
1400,Las Pelotas,Las Pelotas is an Argentinian rock band that o...,83030,1429427,"['Divididos', 'Patricio Rey y sus Redonditos d..."
1401,Mustafá Yoda,Gustavo Adolfo Aciar is an Argentinian MC from...,392,5563,"['Fianru', 'Orion XL', 'Veeyam', 'DToke', '995']"
1402,Audioslave,Audioslave was an American hard rock supergrou...,2794388,72632676,"['Chris Cornell', 'Soundgarden', 'Temple of th..."


In [64]:
# Convertir el 'df_canciones'' a una lista de listas utilizando 'values.tolist()'.
# Cada fila del DataFrame se convierte en una sublista, lo que nos permite trabajar con los datos en un formato de lista.
lista_canciones = df_canciones.values.tolist()
lista_canciones

[['Caminando por la vida',
  'Melendi',
  'pop',
  'album',
  2005,
  '3v1dCP3hk2djfWryqfp7sx'],
 ['Por la boca vive el pez',
  'Fito y Fitipaldis',
  'pop',
  'album',
  2006,
  '24UStI5AYA7g5ufFDh3LH0'],
 ['Soldadito marinero',
  'Fito y Fitipaldis',
  'pop',
  'album',
  2004,
  '0eVborSuxUeSg0meWYd9dZ'],
 ['No Puedo Vivir Sin Ti',
  'Los Ronaldos',
  'pop',
  'album',
  2007,
  '1EC0JRoSi9YL5zExd3Euc3'],
 ['Rosas',
  'La Oreja de Van Gogh',
  'pop',
  'album',
  2003,
  '4waqcUQWdj0yH26STWl2Rq'],
 ['Princesas', 'Pereza', 'pop', 'album', 2005, '3hlWUNjAlbZ6nOjAbrivBv'],
 ['La casa por el tejado',
  'Fito y Fitipaldis',
  'pop',
  'album',
  2004,
  '428cFrawSqeLoyooQjAW9d'],
 ['Marta, Sebas, Guille y los demás',
  'Amaral',
  'pop',
  'album',
  2005,
  '30ddpolNHwXlrS5XO8MiiO'],
 ['Con la luna llena',
  'Melendi',
  'pop',
  'album',
  2006,
  '5VwYcCKNAgDqDJ9ah9tDGl'],
 ['Zapatillas',
  'El Canto Del Loco',
  'pop',
  'album',
  2005,
  '42VgiRyg0YjtRVLiPFNxPJ'],
 ['Dile', 'Don Om

In [65]:
# Convertir el 'df_artistas' a una lista de listas utilizando 'values.tolist()'.
lista_artistas = df_artistas.values.tolist()
lista_artistas

[['Nina Simone',
  'Eunice Kathleen Waymon (21 February 1933 – 21 April 2003), better known by her stage name Nina Simone, was an American singer, pianist, arranger, and civil rights activist widely associated with jazz music. Simone aspired to become a classical pianist while working in a broad range of styles including classical, jazz, blues, soul, folk, rhythm and blues, gospel, and pop.\n\nSimone was born Eunice Kathleen Waymon on 21st February 1933 in Tryon, North Carolina, USA, one of eight children. <a href="https://www.last.fm/music/Nina+Simone">Read more on Last.fm</a>',
  2587613,
  61048146,
  "['Aretha Franklin', 'Billie Holiday', 'Etta James', 'Ella Fitzgerald', 'Sarah Vaughan']"],
 ['Chevelle',
  'Chevelle are an alternative metal band from Grayslake, IL. They formed in 1994 with the members of Sam Loeffler, Pete Loeffler (brothers) and Matt Scott. Matt was later replaced in 1996 by Joe Loeffler as bass player. On July 10, 2005 Joe was fired because of "irreconcilable dif

In [66]:
# Extraer los ids.
ids = [item[-1] for item in lista_canciones]

# Comprobar duplicados.
duplicados = set([id_ for id_ in ids if ids.count(id_) > 1])

# Si hay duplicados, eliminarlos de la lista original.
if duplicados:
    lista_canciones = [item for item in lista_canciones if item[-1] not in duplicados]
    print(f"Ids que se han encontrado duplicados y se han eliminado: {duplicados}")
    print(f"Lista actualizada de canciones sin duplicados: {lista_canciones}")
else:
    print("No hay ids duplicados.")

Ids que se han encontrado duplicados y se han eliminado: {'0SaKhiBPgdRaQKA7Bxu3ek', '1zLJEnxuZBgPbwMrDMiZAf', '1vgfNhZVIU37PVkLXSuek6', '2awKQgW88iq9jFM54VaCAp', '3bJBWsVNUJ3hOIO6lQUKCj', '2Y2q7PkD7lvuWr8YaZfWuv', '3P0ZKM0sJiwYl6LWAgU9YF', '1jiegRk1i62WYyCcmmCSRB', '2MBUKy0r35H9YWukHZRfQM', '0aAR5HogGoT68EWFbyRFqx', '7cQ7axcCXsa963OluHTWlS', '6FstWkgRdHagqJ6qjDbiK0', '55BClNby0SuB7MHxp1Tdn8', '6pYz8bkZvf84S49VWLRHah', '4RAOI1etsgbh5NP3T5R8rN', '16J6wH4Evm97ta1hDQIYQC', '7AMuS6Uoolwio2phw1aVWZ', '1iccv1diVUfq0cCTU2nSEs', '0D2ZQ5DKyys3UPAfeKpvwp', '4w1lzcaoZ1IC2K5TwjalRP', '1cQGnX96iQcZv2cDDZ5u9w', '1zWoGo3R1tAsgsnyPeZkw6', '5lRzWDEe7UuedU2QPsFg0K', '2EVuI3MRs112PtSFiSdrvF', '5jwyM0a0Na89a0EfYMih4N', '6JRVgGubYFxk9zHPSlZtrL', '2u0E5GIxp8begOEeb7GpxQ', '7MKO7rUZzjw8p87zdZDgqa', '5MVh8LWbag2iactJzVD9BP', '0GgN4MhR5GKn5IcKN0e0rG', '1gd7gJb4AJ71q68pFpanMb', '46Yp4JsZIh8ceg5WBqW1ZB', '1zayWmcFFQPQvCuhTEnOZ7', '7mNErWes3YegbRLgZuILl5', '7Gxe4Ewcp2hRe6XB1U8flo', '3HdKJZzxlRYYKcdj00dQdT', '5fSDX

In [67]:
# Convertimos la lista 'lista_canciones' de vuelta a un DataFrame de pandas.
df_lista_canciones = pd.DataFrame(lista_canciones)

In [68]:
# Se establece la conexión con la base de datos MySQL 'music_stream'.
try:
    # Parámetros necesarios como el usuario, la contraseña, la dirección del servidor (localhost) y el nombre de la base de datos.
    cnx = mysql.connector.connect(user='root', password='AlumnaAdalab', host='127.0.0.1', database='music_stream')
    print("Conexión establecida correctamente.")
# Si ocurre algún error en el proceso de conexión, se detecta y maneja adecuadamente.
except mysql.connector.Error as err:
    # En caso de que el error sea un problema con el nombre de usuario o la contraseña.
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Hay algún problema con tu nombre de usuario o contraseña.")
    # En caso de que el error sea que la base de datos no existe.
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("No existe la base de datos.")
    # En caso de que el error sea cualquier otro.
    else:
        print(err)

Conexión establecida correctamente.


In [69]:
my_cursor = cnx.cursor()

In [71]:
# Queries para añadir los datos a cada tabla de la base de datos creada junto con la ejecución de la misma
# Insercción de los artistas.
query_artistas = ("INSERT INTO Artistas (nombre_artistas, biografia, popularidad, reproducciones, artistas_similares) VALUES (%s,%s,%s,%s,%s)")
my_cursor.executemany(query_artistas,lista_artistas)

# Obtener los ids de los artistas después de la inserción.
my_cursor.execute("SELECT id, nombre_artistas FROM Artistas")
artistas = my_cursor.fetchall()
artistas_dict = {artista[1]: artista[0] for artista in artistas}  # Crear un diccionario con nombre -> id.

# Inserción de canciones con id_artista.
query_canciones = ("INSERT INTO AlbumesyCanciones (nombre_cancion, nombre_artista, genre, type, year, id, id_artista) VALUES (%s,%s,%s,%s,%s,%s)")

#query_canciones = ("INSERT INTO AlbumesyCanciones (nombre_cancion, nombre_artista, genre, type, year, id) VALUES (%s,%s,%s,%s,%s,%s)")

# Añadir el id_artista basado en el nombre_artista de lista_canciones.
lista_canciones_con_id_artista = []
for cancion in lista_canciones:
    nombre_artista = cancion[1]  # Asumiendo que el nombre del artista está en la posición 1.
    id_artista = artistas_dict.get(nombre_artista)
    if id_artista:
        lista_canciones_con_id_artista.append((cancion[0], nombre_artista, cancion[2], cancion[3], cancion[4], cancion[5], id_artista))

# Ejecutar la inserción de las canciones con el id_artista correcto.
my_cursor.executemany(query_canciones, lista_canciones_con_id_artista)

#my_cursor.executemany(query_canciones, lista_canciones)

InterfaceError: Failed executing the operation; Not all parameters were used in the SQL statement

In [49]:
# Subir los datos implementados en la base de datos hechos hasta el momento.
cnx.commit()

In [50]:
# Cerrar la conexión con la base de datos.
cnx.close()